In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

In [2]:
driver = webdriver.Chrome()

In [3]:
def get_game_team_stats(html):
    
    #try to develop it to understand how use it without go throw driver from selenium
    
    """
    Args: html from driver.page_source.encode('utf-8')
    
    Return: A list in which each element correspond to player statistics during the game
    
    """
    soup = BeautifulSoup(html)
    teams_box = soup.find_all('section', 'mb-4 overflow-hidden bg-white shadow-block md:mb-6 md:rounded')
    a = []
    for team_box in teams_box:
        team_name = team_box.find('span', 'block').text
        player_stats = team_box.find('tbody')
        players = player_stats.find_all('tr')
        for i, player in enumerate(players): 
            b = [team_name]
            stats = player.find_all('td')
            #just take the row TOTALS of the game for each team (the only of the table with a tag 'block xxx')
            if len(player.find_all('span', 'block py-2 text-xs leading-none')) > 0:
                for i, stat in enumerate(stats):
                    if i >= 2:
                        b.append(stat.text)
                a.append(b)
    return a

In [4]:
team_game_attributes = ['TEAM', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS', '+/-']

In [5]:
url = 'https://www.nba.com/game/det-vs-ind-0021300356/box-score#box-score'
driver.get(url)
html = driver.page_source.encode('utf-8')
a = get_game_team_stats(html)
import numpy as np
a = np.array(a)
import pandas as pd 

df_test = pd.DataFrame(a, columns = team_game_attributes)
df_test

,TEAM,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,+/-
0,Detroit Pistons,38,88,43.2,7,20,35.0,18,28,64.3,20,35,55,14,3,7,8,20,101,5
1,Indiana Pacers,35,81,43.2,6,17,35.3,20,27,74.1,9,31,40,18,5,4,7,20,96,-5


In [6]:
def retrieve_box_score_link2(html):
    """
    Args: html of driver.get(URL) follow by driver.page_source.encode of the URL of the game day, 
    maybe we can change it with just a date as input and create the url
    
    Return: list of link to the box score, the len of the list will be the same as the numnber of
    games during the day
    """
    
    #the watchbox button since the 2012 season change everything 
    
    soup = BeautifulSoup(html)
    game_boxes = soup.find_all('div', 'shadow-block bg-white flex md:rounded text-sm relative mb-4')
    links = []
    #if condition check if there was at least one game this day
    if len(game_boxes) > 0:
        for game_box in game_boxes:
            link = game_box.find('a',
                    {"class": 'block py-3 text-xs font-bold text-center text-cerulean Anchor_complexLink__2NtkO'})['href']
            #we get a useless link which look like this '/game/mia-vs-bos-0021000001/box-score#box-score'
            #to make it useful we just have to add the string 'https://nba.com'
            link = link.split('?')[0]
            link = link + '/box-score'
            links.append('https://nba.com' + link)
        return links
           

In [10]:
URL = 'https://www.nba.com/games?date=2012-10-13'
driver.get(URL)
html = driver.page_source.encode('utf-8')
soup = BeautifulSoup(html)
retrieve_box_score_link2(html)

['https://nba.com/game/bkn-vs-phi-0011200037/box-score',
 'https://nba.com/game/nyk-vs-bos-0011200039/box-score',
 'https://nba.com/game/was-vs-cle-0011200040/box-score',
 'https://nba.com/game/chi-vs-min-0011200038/box-score',
 'https://nba.com/game/det-vs-mil-0011200041/box-score',
 'https://nba.com/game/uta-vs-lal-0011200042/box-score',
 'https://nba.com/game/lac-vs-mia-0011200043/box-score']

In [19]:
URL = 'https://www.nba.com/games?date=2011-12-16'
driver.get(URL)
html = driver.page_source.encode('utf-8')
soup = BeautifulSoup(html)
retrieve_box_score_link2(html)

['https://nba.com/game/chi-vs-ind-0011100001/box-score#box-score/box-score',
 'https://nba.com/game/phi-vs-was-0011100003/box-score#box-score/box-score',
 'https://nba.com/game/cle-vs-det-0011100004/box-score#box-score/box-score',
 'https://nba.com/game/noh-vs-mem-0011100002/box-score#box-score/box-score']

Test looping over one game day

In [35]:
import time
#we suppose that a Selenium driver is open before
URL = 'https://www.nba.com/games?date=2012-02-01'
driver.get(URL)
html = driver.page_source.encode('utf-8')
soup = BeautifulSoup(html)
links = retrieve_box_score_link2(html)
a = []
id_ = 0
for link in links:
    driver.get(link)
    #have to put a sleeper to be sure to load the website
    #FIND A BETTER WAY TO DEAL WITH THIS
    time.sleep(2)
    html_boxscore = driver.page_source.encode('utf-8')
    team_box_score = get_game_team_stats(html_boxscore)
    for i, team_stats in enumerate(team_box_score):
        print(i)
        if i == 0 :
            team_stats.append('0')
        else : 
            team_stats.append('1')
        team_stats.append(str(id_))
        a.append(team_stats)
    id_ += 1
a

0
['Chicago Bulls', '32', '74', '43.2', '7', '17', '41.2', '11', '16', '68.8', '8', '29', '37', '22', '5', '5', '17', '15', '82', '-16', '0', '0']
1
['Philadelphia 76ers', '40', '84', '47.6', '4', '11', '36.4', '14', '19', '73.7', '11', '32', '43', '26', '10', '4', '10', '14', '98', '16', '1', '0']
0
['Washington Wizards', '42', '92', '45.7', '12', '23', '52.2', '7', '9', '77.8', '15', '30', '45', '26', '8', '5', '16', '28', '103', '-6', '0', '1']
1
['Orlando Magic', '33', '73', '45.2', '13', '30', '43.3', '30', '39', '76.9', '10', '27', '37', '18', '8', '1', '15', '10', '109', '6', '1', '1']
0
['Toronto Raptors', '25', '73', '34.2', '6', '14', '42.9', '8', '18', '44.4', '10', '29', '39', '16', '4', '4', '15', '19', '64', '-36', '0', '2']
1
['Boston Celtics', '39', '75', '52.0', '12', '24', '50.0', '10', '12', '83.3', '6', '40', '46', '30', '4', '6', '10', '20', '100', '36', '1', '2']
0
['Detroit Pistons', '40', '82', '48.8', '5', '18', '27.8', '11', '14', '78.6', '11', '20', '31', '18

KeyboardInterrupt: 

In [7]:
import numpy as np
import pandas as pd
import time

In [8]:
def get_season_url(season, begin_month, begin_day, end_month, end_day):
    """
    Args: 
        season: string the year when the season started
        begin_month: string month when the season started, month should be '01', '02', ..., '12'
        begin_day: string day of the month when the season started
    Return: list of every URL game date for all day during the regular season
    """
    path = "https://www.nba.com/games?date="
    
    #initialization of months and day in each months
    
    #IF THE SEASON STARTS IN JANUARY THERE IS AN ERROR!!
    months = []
    for k in [10, 11, 12, 1, 2, 3, 4, 5, 6]:
        if k < 10:
            months.append('0' + str(k))
        else:
            if k >= int(begin_month):
                months.append(str(k))
    day_even = []
    for k in range(1, 31):
        if k < 10:
            day_even.append('0' + str(k))
        else:
            day_even.append(str(k))
    day_odd = []
    for k in range(1, 32):
        if k < 10:
            day_odd.append('0' + str(k))
        else:
            day_odd.append(str(k))
    
    link_list = []
    for month in months:
        if month == begin_month :
            for day in day_odd[(int(begin_day) - 1):] :
                date_ = season + '-' + month + '-' + day
                link_list.append(path + date_)
            continue
        if month == end_month :
            for day in day_even[:int(end_day)] :
                date_ = season + '-' + month + '-' + day
                link_list.append(path + date_)
            break
        if month == '02':
            #check is the year is bisextile
            if (int(season) % 4 == 0) and (int(season)% 10 != 0):
                for day in day_odd[:29]:
                    date_ = season + '-' + month + '-' + day
                    link_list.append(path + date_)
            else :
                for day in day_odd[:28]:
                    date_ = season + '-' + month + '-' + day
                    link_list.append(path + date_)      
        if month in ['01', '03', '05', '07', '08', '10', '12']:
            for day in day_odd :
                date_ = season + '-' + month + '-' + day
                link_list.append(path + date_)
            if month == '12':
                aux = int(season) + 1
                season = str(aux)
        if month in ['04', '06', '09', '11']:
            for day in day_even:
                date_ = season + '-' + month + '-' + day
                link_list.append(path + date_)

    return link_list 

In [9]:
s10 = get_season_url('2010', '10', '26', '04', '13')
s11 = get_season_url('2011', '12', '25', '04', '26')
s12 = get_season_url('2012', '10', '30', '04', '17')
s13 = get_season_url('2013', '10', '29', '04', '16')
s14 = get_season_url('2014', '10', '28', '04', '15')
s15 = get_season_url('2015', '10', '27', '04', '13')
s16 = get_season_url('2016', '10', '25', '04', '12')
s17 = get_season_url('2017', '10', '17', '04', '11')
s18 = get_season_url('2018', '10', '16', '04', '13')
s20 = get_season_url('2020', '12', '22', '05', '16')
S = [s10, s11, s12, s13, s14, s15, s16, s17, s18, s20]

In [10]:
s16[:10]

['https://www.nba.com/games?date=2016-10-25',
 'https://www.nba.com/games?date=2016-10-26',
 'https://www.nba.com/games?date=2016-10-27',
 'https://www.nba.com/games?date=2016-10-28',
 'https://www.nba.com/games?date=2016-10-29',
 'https://www.nba.com/games?date=2016-10-30',
 'https://www.nba.com/games?date=2016-10-31',
 'https://www.nba.com/games?date=2016-11-01',
 'https://www.nba.com/games?date=2016-11-02',
 'https://www.nba.com/games?date=2016-11-03']

In [11]:
import time

In [ ]:
id_ = 0
a = []
for i, url in enumerate(s16):
    driver.get(url)
    time.sleep(4)
    html = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(html)
    links = retrieve_box_score_link2(html)
    date = url.split('=')[1]
    if links != None:
        for link in links:
            driver.get(link)
            #have to put a sleeper to be sure to load the website
            #FIND A BETTER WAY TO DEAL WITH THIS
            time.sleep(6)
            html_boxscore = driver.page_source.encode('utf-8')
            team_box_score = get_game_team_stats(html_boxscore)
            for i, team_stats in enumerate(team_box_score):
                if i == 0 :
                    team_stats.append('0')
                else : 
                    team_stats.append('1')
                team_stats.append(str(id_))
                a.append(team_stats)
            id_ += 1
a

In [29]:
url

'https://www.nba.com/games?date=2016-04-13'

In [30]:
team_game_attributes = ['TEAM', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS', '+/-', 'HOME', 'GAME-ID']

In [31]:
a = np.array(a)
df = pd.DataFrame(a, columns = team_game_attributes)
df

,TEAM,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,...,REB,AST,STL,BLK,TO,PF,PTS,+/-,HOME,GAME-ID
0,Detroit Pistons,37,96,38.5,12,29,41.4,20,26,76.9,...,59,23,5,3,15,15,106,12,0,0
1,Atlanta Hawks,37,82,45.1,8,27,29.6,12,15,80.0,...,40,22,9,4,15,25,94,-12,1,0
2,Cleveland Cavaliers,38,94,40.4,9,29,31.0,10,17,58.8,...,50,26,5,7,10,21,95,-2,0,1
3,Chicago Bulls,37,87,42.5,7,19,36.8,16,23,69.6,...,47,13,6,10,13,22,97,2,1,1
4,New Orleans Pelicans,35,83,42.2,6,18,33.3,19,27,70.4,...,33,21,9,3,18,26,95,-16,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2441,Los Angeles Lakers,41,85,48.2,6,25,24.0,13,15,86.7,...,47,19,6,3,13,17,101,5,1,1230
2442,LA Clippers,43,96,44.8,9,31,29.0,10,12,83.3,...,40,23,9,6,15,20,105,-9,0,1231
2443,Phoenix Suns,45,99,45.5,7,26,26.9,17,22,77.3,...,58,26,10,5,16,15,114,9,1,1231
2444,Denver Nuggets,40,94,42.6,7,25,28.0,12,18,66.7,...,53,24,10,6,20,19,99,-8,0,1232


In [32]:
df.to_csv('team_stats_2015-2016.csv')

In [68]:
url

'https://www.nba.com/games?date=2015-04-12'